In [1]:
import pandas as pd
import altair as alt
from json import loads

G = 1024 * 1024 * 1024
M = 1024 * 1024

In [8]:
df = pd.read_json("../profiling/results/report.3.json", lines=True)
df["DVM"] = df["mem_size"] / df["memory"]
df["MachineMem"] = df["memory"].apply(lambda value: f"{value / G:.0f}G")
df["DatasetMemSize"] = df["mem_size"].apply(lambda value: f"{value / M:.1f}M")

In [9]:
df

,cpu,dataset,fname,format,memory,ncol,nrow,partition,reader,mem_size,name,elapsed,DVM,MachineMem,DatasetMemSize
0,8,adult,create_report_pp.py,csv,8589934592,0,0,0,pandas,32319408,PandasProfilingReport,28.243927,0.003762,8G,30.8M
1,8,adult,create_report_dp.py,csv,8589934592,0,0,0,pandas,32319408,DataPrepReport,8.239377,0.003762,8G,30.8M
2,8,automobile,create_report_pp.py,csv,8589934592,0,0,0,pandas,219841,PandasProfilingReport,49.003691,0.000026,8G,0.2M
3,8,automobile,create_report_dp.py,csv,8589934592,0,0,0,pandas,219841,DataPrepReport,5.260094,0.000026,8G,0.2M
4,8,covid19_outcome,create_report_pp.py,csv,8589934592,0,0,0,pandas,3424458,PandasProfilingReport,16.620765,0.000399,8G,3.3M
5,8,covid19_outcome,create_report_dp.py,csv,8589934592,0,0,0,pandas,3424458,DataPrepReport,2.185981,0.000399,8G,3.3M
6,8,heart_failure,create_report_pp.py,csv,8589934592,0,0,0,pandas,36008,PandasProfilingReport,25.124970,0.000004,8G,0.0M
7,8,heart_failure,create_report_dp.py,csv,8589934592,0,0,0,pandas,36008,DataPrepReport,2.826334,0.000004,8G,0.0M
8,8,suicide,create_report_pp.py,csv,8589934592,0,0,0,pandas,12979530,PandasProfilingReport,24.179507,0.001511,8G,12.4M
9,8,suicide,create_report_dp.py,csv,8589934592,0,0,0,pandas,12979530,DataPrepReport,3.641172,0.001511,8G,12.4M


In [12]:
(
    alt.Chart(df[~df.dataset.isin(["yelp_academic_dataset_review"])], title="create_report(df) Comparison")
    .mark_bar()
    .encode(
        y=alt.Y("name", title=""),
        x=alt.X("elapsed", title="Elapsed (s)"),
        color=alt.Color("name:N", legend=None),
        tooltip=[
            alt.Tooltip("name:N"),
            alt.Tooltip("elapsed:Q", title="Elapsed (s)", format=".2s"),
            alt.Tooltip("MachineMem"),
            alt.Tooltip("DatasetMemSize"),
        ],
        row=alt.Row("dataset", title="Dataset", header=alt.Header(labelAngle=0, labelAlign="left")),
    )
)

alt.Chart(...)

In [ ]:
##
chartDVM = (
    alt.Chart(df[df["DVM"] < 10], title="plot_correlation(df) Comparison")
    .mark_line(point=True)
    .encode(
        y=alt.Y("elapsed", title="Elapsed (s)"),
        x=alt.X("DVM", title="Dataset Size / Memory Size"),
        color="name:N",
        tooltip=[
            alt.Tooltip("name:N"),
            alt.Tooltip("elapsed:Q", format=".0s"),
            alt.Tooltip("MachineMem"),
            alt.Tooltip("DatasetMemSize"),
        ],
        column=alt.Column("format", title="Data Format"),
        row=alt.Column("reader", title="Data Reader"),
    )
)
chartDVM
##
# chartBar = (
#     alt.Chart(
#         df[df.memory == 1 * G], title="Plot Comparison: 8G Mem/8 CPU/16 Data Partition"
#     )
#     .mark_bar()
#     .encode(
#         y="name:N",
#         x=alt.X("elapsed", title="Elapsed (s)"),
#         color="name",
#         tooltip="elapsed",
#         row="nrow:Q",
#         column=alt.Column("format:O", title="Data Format"),
#     )
#     .resolve_scale(x="independent")
# )
# chartBar
##
# pdf = df.pivot_table(
#     index=["Mem", "CPU", "Dataset", "Partition", "Row", "Col", "Mode"],
#     columns="Func",
#     values="Elapsed",
# ).reset_index()


##
